# 🏥 Cardiovascular Disease Dataset - Quick Start Guide

This notebook demonstrates how to load and explore the cardiovascular disease longitudinal dataset.

## 📊 Dataset Overview
- **3,000 patients** tracked over **8 time points**
- **Static features**: Demographics, risk factors
- **Dynamic features**: Symptoms, labs, treatments, outcomes
- **Ground truth**: Disease states for supervised learning

In [ ]:
# Import required libraries
import sys
import os

# Add src to path for imports
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import our project utilities
from project import CVDDataLoader, CVDVisualizer, quick_eda_plots

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Libraries imported successfully!")

## 📂 Load Data

The raw data is now included in the repository for easy access and reproducibility.

In [ ]:
# Initialize data loader
loader = CVDDataLoader(data_path="../data/raw")

# Load raw datasets
patients, encounters = loader.load_raw_data()

print(f"📊 Loaded {len(patients):,} patients and {len(encounters):,} encounters")
print(f"⏱️  Average encounters per patient: {len(encounters) / len(patients):.1f}")

## 🔍 Data Structure Overview

In [ ]:
# Patient data structure
print("👥 PATIENT DATA")
print("=" * 50)
print(f"Shape: {patients.shape}")
print(f"Columns: {list(patients.columns)}")
print("\nFirst 5 rows:")
display(patients.head())

print("\n" + "=" * 70)

# Encounter data structure
print("🏥 ENCOUNTER DATA")
print("=" * 50)
print(f"Shape: {encounters.shape}")
print(f"Columns: {list(encounters.columns)}")
print("\nFirst 5 rows:")
display(encounters.head())

## 📊 Data Summary Statistics

In [ ]:
# Generate comprehensive data summary
summary = loader.get_data_summary()

print("📈 PATIENT DEMOGRAPHICS")
print("=" * 50)
print(f"Total patients: {summary['patients']['total_count']:,}")
print(f"Age range: {summary['patients']['age_stats']['min']:.0f}-{summary['patients']['age_stats']['max']:.0f} years")
print(f"Age mean: {summary['patients']['age_stats']['mean']:.1f} years")
print(f"\nSex distribution:")
for sex, count in summary['patients']['sex_distribution'].items():
    print(f"  {sex}: {count:,} ({count/summary['patients']['total_count']:.1%})")

print(f"\nRisk factor prevalence:")
print(f"  Smokers: {summary['patients']['smoker_rate']:.1%}")
print(f"  Family history: {summary['patients']['family_history_rate']:.1%}")
print(f"  Hypertension: {summary['patients']['hypertension_rate']:.1%}")

print(f"\nInitial disease states:")
for state, count in summary['patients']['initial_state_distribution'].items():
    print(f"  {state}: {count:,} ({count/summary['patients']['total_count']:.1%})")

print("\n" + "=" * 70)

print("🏥 ENCOUNTER STATISTICS")
print("=" * 50)
print(f"Total encounters: {summary['encounters']['total_count']:,}")
print(f"Encounters per patient: {summary['encounters']['encounters_per_patient']['mean']:.1f}")

print(f"\nDisease state distribution across all visits:")
for state, count in summary['encounters']['state_distribution'].items():
    print(f"  {state}: {count:,} ({count/summary['encounters']['total_count']:.1%})")

print(f"\nTreatment distribution:")
for treatment, count in summary['encounters']['treatment_distribution'].items():
    print(f"  {treatment}: {count:,} ({count/summary['encounters']['total_count']:.1%})")

print(f"\nUtility score: {summary['encounters']['utility_stats']['mean']:.3f} ± {summary['encounters']['utility_stats']['std']:.3f}")

if summary['encounters']['missing_data_rates']:
    print(f"\nMissing data rates:")
    for col, rate in summary['encounters']['missing_data_rates'].items():
        print(f"  {col}: {rate:.1%}")
else:
    print("\n✅ No missing data detected")

## 🔗 Merge Data for Analysis

In [ ]:
# Merge patient and encounter data
merged_data = loader.merge_data(include_time_features=True)

print(f"📊 Merged dataset shape: {merged_data.shape}")
print(f"📋 New columns added: {[col for col in merged_data.columns if col not in encounters.columns and col not in patients.columns]}")

# Show sample of merged data
print("\nSample merged data (first patient):")
sample_patient = merged_data[merged_data['patient_id'] == 1]
display(sample_patient[['patient_id', 'time', 'state', 'treatment', 'age', 'sex', 
                       'risk_score', 'utility', 'is_baseline', 'is_final']].head())

## 📈 Generate Comprehensive EDA Visualizations

Let's create a complete set of exploratory data analysis plots.

In [ ]:
# Generate all EDA plots at once
figures = quick_eda_plots(patients, encounters)

# Display plots
for i, fig in enumerate(figures):
    print(f"\n{'='*60}")
    print(f"Figure {i+1}: {fig._suptitle.get_text() if fig._suptitle else 'Visualization'}")
    print(f"{'='*60}")
    plt.show()

## 🎯 Sample Patient Trajectories

Let's examine individual patient journeys through the healthcare system.

In [ ]:
# Get sample patient trajectories
sample_trajectories = loader.get_patient_trajectories(n_sample=5)

print("🎯 SAMPLE PATIENT TRAJECTORIES")
print("=" * 60)

for patient_id in sample_trajectories['patient_id'].unique():
    patient_data = sample_trajectories[sample_trajectories['patient_id'] == patient_id]
    
    print(f"\n👤 Patient {patient_id}:")
    print(f"   Demographics: {patient_data.iloc[0]['age']}y {patient_data.iloc[0]['sex']}, "
          f"BMI {patient_data.iloc[0]['bmi']:.1f}, Risk Score {patient_data.iloc[0]['risk_score']}")
    print(f"   Baseline State: {patient_data.iloc[0]['initial_state']}")
    
    print("   Journey:")
    for _, encounter in patient_data.iterrows():
        treatment_str = f"→ {encounter['treatment']}" if encounter['treatment'] != 'None' else ""
        print(f"     T{encounter['time']}: {encounter['state']} {treatment_str} (Utility: {encounter['utility']:.3f})")

print("\n" + "=" * 60)
print("💡 Key Observations:")
print("   • Disease states can progress: Healthy → Early → Advanced")
print("   • Treatments may stabilize or improve outcomes")
print("   • Utility scores reflect patient quality of life")
print("   • Each patient follows a unique trajectory")

## 🧮 Basic Statistical Tests

In [ ]:
from scipy import stats

print("🧮 STATISTICAL INSIGHTS")
print("=" * 50)

# Test: Age difference by sex
male_ages = patients[patients['sex'] == 'M']['age']
female_ages = patients[patients['sex'] == 'F']['age']
age_ttest = stats.ttest_ind(male_ages, female_ages)

print(f"📊 Age by Sex:")
print(f"   Males: {male_ages.mean():.1f} ± {male_ages.std():.1f} years (n={len(male_ages)})")
print(f"   Females: {female_ages.mean():.1f} ± {female_ages.std():.1f} years (n={len(female_ages)})")
print(f"   T-test p-value: {age_ttest.pvalue:.4f}")

# Test: Utility by treatment
treatment_groups = []
for treatment in encounters['treatment'].unique():
    group_utility = encounters[encounters['treatment'] == treatment]['utility']
    treatment_groups.append(group_utility)
    print(f"\n💊 {treatment} Utility: {group_utility.mean():.3f} ± {group_utility.std():.3f} (n={len(group_utility)})")

# ANOVA for treatment effects
treatment_anova = stats.f_oneway(*treatment_groups)
print(f"\n🔬 Treatment Effect ANOVA:")
print(f"   F-statistic: {treatment_anova.statistic:.3f}")
print(f"   P-value: {treatment_anova.pvalue:.6f}")

# Correlation between risk score and outcomes
baseline_data = merged_data[merged_data['time'] == 0]
risk_utility_corr = stats.pearsonr(baseline_data['risk_score'], baseline_data['utility'])
print(f"\n📈 Risk Score vs Baseline Utility:")
print(f"   Correlation: {risk_utility_corr[0]:.3f}")
print(f"   P-value: {risk_utility_corr[1]:.6f}")

## 💾 Save Processed Data

In [ ]:
# Save processed datasets for future use
import os

# Create processed data directory if it doesn't exist
processed_dir = "../data/processed"
os.makedirs(processed_dir, exist_ok=True)

# Save merged dataset
merged_data.to_csv(f"{processed_dir}/merged_data.csv", index=False)
print(f"✅ Saved merged dataset: {processed_dir}/merged_data.csv")

# Save summary statistics
summary_df = pd.DataFrame({
    'metric': ['total_patients', 'total_encounters', 'avg_encounters_per_patient', 
               'avg_age', 'pct_male', 'pct_smokers', 'avg_utility'],
    'value': [summary['patients']['total_count'], 
              summary['encounters']['total_count'],
              summary['encounters']['encounters_per_patient']['mean'],
              summary['patients']['age_stats']['mean'],
              summary['patients']['sex_distribution'].get('M', 0) / summary['patients']['total_count'],
              summary['patients']['smoker_rate'],
              summary['encounters']['utility_stats']['mean']]
})
summary_df.to_csv(f"{processed_dir}/data_summary.csv", index=False)
print(f"✅ Saved data summary: {processed_dir}/data_summary.csv")

print(f"\n📊 Data is ready for analysis!")
print(f"   • Use 'merged_data' DataFrame for modeling")
print(f"   • Files saved in {processed_dir}/")
print(f"   • Next steps: Feature engineering, model training, evaluation")

## 🚀 Next Steps

Now that you have loaded and explored the data, you can proceed with:

1. **Feature Engineering** (M1):
   - Create temporal features (lag variables, rolling statistics)
   - Derive clinical risk scores
   - Handle missing data strategically

2. **Model Development** (M1-M2):
   - Classification: Predict disease state progression
   - Time series: Forecast future clinical values
   - Clustering: Identify patient subgroups

3. **Advanced Modeling** (M2-M3):
   - Hidden Markov Models for state transitions
   - Bayesian Networks for causal modeling
   - Reinforcement Learning for treatment optimization

4. **Evaluation & Interpretation** (M3):
   - Model performance across patient subgroups
   - Clinical relevance of predictions
   - Treatment policy recommendations

---

**📝 Documentation**: See `data/raw/README.md` for detailed data documentation

**🔧 Code**: Reusable functions are in `src/project/` for import across notebooks

**📊 Milestones**: Check `milestones/Mx/` for specific deliverables and timelines